In [38]:
PNG_PATH = 'C:\\Users\\user\\Downloads\\deep_scores_dense\\deep_scores_dense\\images_png'
XML_PATH = 'C:\\Users\\user\\Downloads\\deep_scores_dense\\deep_scores_dense\\xml_annotations'
CONVERTED_PATH = 'C:\\Users\\user\\Downloads\\deep_scores_dense\\deep_scores_dense\\converted'
CLASSES_PATH = 'C:\\Users\\user\\Downloads\\deep_scores_dense\\deep_scores_dense\\meta_info'
X = 100
Y = 100
PARTS_NUMBER = 100
ELEMENTS_MAX_NUMBER = 10

In [ ]:
import os
import imageio as im
import random
import numpy as np
from dataclasses import dataclass
import xml.etree.ElementTree as ET

random.seed()

@dataclass
class PartData:
    slice: list
    y: int
    x: int
    max_y: int
    max_x: int

def split_image(filename):
    
    image = np.array(im.imread(PNG_PATH + "\\" + filename))
    shape = np.shape(image)
    max_y = shape[0] - Y
    max_x = shape[1] - X
    parts = []
    
    for i in range(PARTS_NUMBER):
        random_y = random.randrange(max_y)
        random_x = random.randrange(max_x)
        slice = image[random_y:random_y + Y, random_x:random_x + X, 0]
        parts.append(PartData(slice, random_y, random_x, shape[0], shape[1]))
        
    return parts

In [48]:
@dataclass
class BoundingBox:
    xmin: float
    xmax: float
    ymin: float
    ymax: float
        
    def is_within(self, bb):
        return bb.xmin <= self.xmin and bb.xmax >= self.xmax and bb.ymin <= self.ymin and bb.ymax >= self.ymax

    def get_normalized(self, x, y):
        return BoundingBox(self.xmin/x, self.xmax/x, self.ymin/y, self.ymax/y)

    def serialize(self):
        return np.array(self.xmin, self.xmax, self.ymin, self.ymax)

def retrieve_class_names():
    files = os.listdir(CLASSES_PATH)
    counter = 0
    names = {}
    for f in files:
        names[f.replace('.csv', '')] = counter
        counter += 1
    return names

classes = retrieve_class_names()

@dataclass
class Element:
    typ: str
    bounding_box: BoundingBox   

    def serlialize(self):
        prob_list = np.zeros([len(classes)])
        prob_list[classes[self.typ]] = 1.0
        return np.array(prob_list, self.bounding_box.serialize(), list(1))

def serialize_element_list(elements):
    output_array = []
    for i in range(ELEMENTS_MAX_NUMBER - len(elements)):
        elements.append(
            Element()
        )
    for e in elements:
        output_array.append(e.serialize)
    return np.array(output_array)

class Page:

    def __init__(self, name):
        self.name = name
        self.elements = []
        
    def add(self, element):
        self.elements.append(element)
        
    def retrieve_from_box(self, bb):
        filtered = []
        for e in self.elements:
            if e.is_within(bb):
                filtered.append(e)
        return filtered

    
class Retriever:
    
    def retrieve():

        files = os.listdir(XML_PATH)
        pages = {}
        num_of_files = len(files)
        counter = 1

        for f in files:

            if counter % 100 == 0:
                print(f'{round(counter*100/num_of_files)}% of the files processed, we are at {counter} in {num_of_files}.')
            counter += 1
            page = Page(f)
            tree = ET.parse(XML_PATH + "\\" + f)
            root = tree.getroot()

            for obj in root.iter('object'):

                raw_bb = obj.find('bndbox')
                page.add(BoundingBox(
                    float(raw_bb.find('xmin').text),
                    float(raw_bb.find('xmax').text),
                    float(raw_bb.find('ymin').text),
                    float(raw_bb.find('ymax').text)
                ))

            pages[f] = page

        return pages         

In [ ]:
pages = Retriever.retrieve() 

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [33]:
# Get generator of da

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

@dataclass
class Data:
    img: list
    elements: list

    def to_ts(self):
        return (self.img, serialize_element_list(self.elements))

def generator(files):
    for file in files:
        splitted = split_image(file)
        page = pages[file.replace(".png", ".xml")]
        for s in splitted:
            data = Data(s.slice, page.retrieve_from_box(BoundingBox(s.x / s.max_x, (s.x + X) / s.max_x, s.y / s.max_y, (s.y + Y) / s.max_y)))
            yield data.to_ts()

In [34]:
files = os.listdir(PNG_PATH)
ds = tf.data.Dataset.from_generator(generator, args=files, output_types=(tf.int32, tf.)


TypeError: Either `output_signature` or `output_types` must be specified